In [1]:
import tensorflow as tf

def categorical_loss(F,Y):
    return -tf.math.reduce_mean(tf.math.log(tf.reduce_sum(F*Y,1)),0)

model=tf.keras.models.load_model("NetWorkInNetwork_4_Blocks.model", custom_objects={'loss_kev': categorical_loss})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
new_model = tf.keras.models.Sequential()

for i in range(22):
    layer = model.get_layer(index=i)
    layer.trainable = False
    new_model.add(layer)

In [3]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 192)       14400     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32, 32, 192)       768       
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 160)       30720     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 32, 32, 160)       640       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 160)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        15360     
__________

In [4]:
import numpy as np

my_init_4=tf.keras.initializers.RandomNormal(mean=0.0, stddev=np.math.sqrt(2.0/192))
my_init_5=tf.keras.initializers.RandomNormal(mean=0.0, stddev=np.math.sqrt(2.0/(3*3*192)))
my_init_dense=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='uniform')

new_model.add(tf.keras.layers.Conv2D(192,kernel_size=3,strides=[1,1], kernel_initializer=my_init_5,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv1'))
new_model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn1'))
new_model.add(tf.keras.layers.ReLU(name='relu1'))
new_model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv2'))
new_model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn2'))
new_model.add(tf.keras.layers.ReLU(name='relu2'))
new_model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv3'))
new_model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn3'))
new_model.add(tf.keras.layers.ReLU(name='relu3'))

new_model.add(tf.keras.layers.Flatten())
new_model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
new_model.add(tf.keras.layers.BatchNormalization(beta_initializer='zeros', gamma_initializer='ones', name='bn4'))
new_model.add(tf.keras.layers.ReLU(name='relu4'))
new_model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
new_model.add(tf.keras.layers.BatchNormalization(beta_initializer='zeros', gamma_initializer='ones', name='bn5'))
new_model.add(tf.keras.layers.ReLU(name='relu5'))
new_model.add(tf.keras.layers.Dense(4, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4), bias_initializer='zeros',bias_regularizer=tf.keras.regularizers.l2(5e-4)))

In [5]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 192)       14400     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32, 32, 192)       768       
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 160)       30720     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 32, 32, 160)       640       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 160)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        15360     
__________

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

f = open('TrafficDataset/coordinates.txt', 'r')
lines=f.readlines()

NUM_TEST = round(0.15*len(lines)) # 85% train, 15% test

x_train = np.zeros((len(lines)-NUM_TEST, 32, 32, 3))
y_train = np.zeros((len(lines)-NUM_TEST, 4))
x_test = np.zeros((NUM_TEST, 32, 32, 3))
y_test = np.zeros((NUM_TEST, 4))

for i in range(len(lines)-NUM_TEST):
    line = lines[i]
    coords = [int(j) for j in line.split(';')[1:]]
    img_path = 'TrafficDataset/'+str(i).zfill(6)+'.ppm'
    img = cv2.imread(img_path)
    x_train[i] = img
    y_train[i,:] = coords

for i in range(NUM_TEST):
    line = lines[len(lines)-NUM_TEST+i]
    coords = [int(j) for j in line.split(';')[1:]]
    img_path = 'TrafficDataset/'+str(len(lines)-NUM_TEST+i).zfill(6)+'.ppm'
    img = cv2.imread(img_path)
    x_test[i] = img
    y_test[i,:] = coords

# Number of training examples
n_train = len(x_train) 

# Number of testing examples.
n_test = len(x_test) 

# What's the shape of the image
image_shape = x_train[0].shape 

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)

def normalize(img,img2):
    img_array = np.asarray(img)
    img_array2 = np.asarray(img2)
    normalized = np.empty(np.shape(img_array))
    normalized2 = np.empty(np.shape(img_array2))
    for i in range(3):
        normalized = (img_array - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
        normalized2 = (img_array2 - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
    return normalized,normalized2

x_train,x_test=normalize(x_train,x_test)

3639
Number of training examples = 3093
Number of testing examples = 546
Image data shape = (32, 32, 3)


In [7]:
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(epoch):
    initial_lrate = 0.05
    if epoch<20:
        lrate = initial_lrate
    elif epoch<40:
        lrate = initial_lrate/5
    elif epoch<45:
        lrate = initial_lrate/25
    elif epoch<50:
        lrate = initial_lrate/(25*5)
    else:
        lrate = initial_lrate/(25*25)
    return lrate

loss_history = LossHistory()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

my_optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)

new_model.compile(optimizer=my_optimizer,loss='mean_squared_error',metrics=['accuracy'])
new_model.fit(x_train,y_train, batch_size=16,epochs=100, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
3093/3093 [==============================] - 6s 2ms/sample - loss: 76.5266 - acc: 0.4336
Epoch 2/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 63.4181 - acc: 0.5015
Epoch 3/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 54.3719 - acc: 0.5571
Epoch 4/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 47.8234 - acc: 0.5558
Epoch 5/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 43.2440 - acc: 0.5645
Epoch 6/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 40.8508 - acc: 0.5684
Epoch 7/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 37.3917 - acc: 0.5703
Epoch 8/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 33.9418 - acc: 0.5648
Epoch 9/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 31.6464 - acc: 0.5917
Epoch 10/100
3093/3093 [===================

In [8]:
val_loss, val_acc=new_model.evaluate(x_test,y_test)
print(val_loss, val_acc)

546/546 [==============================] - 1s 1ms/sample - loss: 17.5466 - acc: 0.6991
17.546561372223625 0.69905216


In [9]:
new_model.save("traffic_sign.model")